In [1]:
import qstock as qs
from qstock import plot
from datetime import *
import exchange_calendars as xcals
import numpy as np
import pandas as pd
from datetime import timedelta, datetime
import time as _time
from datetime import *
from pandas import DataFrame
import plotly_express as px
import plotly.graph_objects as go

date=datetime.today().strftime("%Y-%m-%d")
xshg = xcals.get_calendar("XSHG")
xshg_range = xshg.schedule.loc["2000-01-01":""+date+""]

In [ ]:
# -*- coding: utf-8 -*-
from EmQuantAPI import *
from datetime import timedelta, datetime
import time as _time
import traceback
from datetime import *
import matplotlib as mpl
import matplotlib.pyplot as plt
import xlwings as xw
import xlrd
import numpy as np
import pandas as pd
from pandas import DataFrame
from pylab import mpl
from pandas.plotting import table
import plotly_express as px
import plotly.graph_objects as go

def mainCallback(quantdata):
    """
    mainCallback 是主回调函数，可捕捉如下错误
    在start函数第三个参数位传入，该函数只有一个为c.EmQuantData类型的参数quantdata
    :param quantdata:c.EmQuantData
    :return:
    """
    print ("mainCallback",str(quantdata))
    #登录掉线或者 登陆数达到上线（即登录被踢下线） 这时所有的服务都会停止
    if str(quantdata.ErrorCode) == "10001011" or str(quantdata.ErrorCode) == "10001009":
        print ("Your account is disconnect. You can force login automatically here if you need.")
    #行情登录验证失败（每次连接行情服务器时需要登录验证）或者行情流量验证失败时，会取消所有订阅，用户需根据具体情况处理
    elif str(quantdata.ErrorCode) == "10001021" or str(quantdata.ErrorCode) == "10001022":
        print ("Your all csq subscribe have stopped.")
    #行情服务器断线自动重连连续6次失败（1分钟左右）不过重连尝试还会继续进行直到成功为止，遇到这种情况需要确认两边的网络状况
    elif str(quantdata.ErrorCode) == "10002009":
        print ("Your all csq subscribe have stopped, reconnect 6 times fail.")
    # 行情订阅遇到一些错误(这些错误会导致重连，错误原因通过日志输出，统一转换成EQERR_QUOTE_RECONNECT在这里通知)，正自动重连并重新订阅,可以做个监控
    elif str(quantdata.ErrorCode) == "10002012":
        print ("csq subscribe break on some error, reconnect and request automatically.")
    # 资讯服务器断线自动重连连续6次失败（1分钟左右）不过重连尝试还会继续进行直到成功为止，遇到这种情况需要确认两边的网络状况
    elif str(quantdata.ErrorCode) == "10002014":
        print ("Your all cnq subscribe have stopped, reconnect 6 times fail.")
    # 资讯订阅遇到一些错误(这些错误会导致重连，错误原因通过日志输出，统一转换成EQERR_INFO_RECONNECT在这里通知)，正自动重连并重新订阅,可以做个监控
    elif str(quantdata.ErrorCode) == "10002013":
        print ("cnq subscribe break on some error, reconnect and request automatically.")
    # 资讯登录验证失败（每次连接资讯服务器时需要登录验证）或者资讯流量验证失败时，会取消所有订阅，用户需根据具体情况处理
    elif str(quantdata.ErrorCode) == "10001024" or str(quantdata.ErrorCode) == "10001025":
        print("Your all cnq subscribe have stopped.")
    else:
        pass

def startCallback(message):
    print("[EmQuantAPI Python]", message)
    return 1
def csqCallback(quantdata):
    """
    csqCallback 是csq订阅时提供的回调函数模板。该函数只有一个为c.EmQuantData类型的参数quantdata
    :param quantdata:c.EmQuantData
    :return:
    """
    print ("csqCallback,", str(quantdata))

def cstCallBack(quantdata):
    '''
    cstCallBack 是日内跳价服务提供的回调函数模板
    '''
    for i in range(0, len(quantdata.Codes)):
        length = len(quantdata.Dates)
        for it in quantdata.Data.keys():
            print(it)
            for k in range(0, length):
                for j in range(0, len(quantdata.Indicators)):
                    print(quantdata.Data[it][j * length + k], " ",end = "")
                print()
def cnqCallback(quantdata):
    """
    cnqCallback 是cnq订阅时提供的回调函数模板。该函数只有一个为c.EmQuantData类型的参数quantdata
    :param quantdata:c.EmQuantData
    :return:
    """
    # print ("cnqCallback,", str(quantdata))
    print("cnqCallback,")
    for code in quantdata.Data:
        total = len(quantdata.Data[code])
        for k in range(0, len(quantdata.Data[code])):
            print(quantdata.Data[code][k])

try:
    #调用登录函数（激活后使用，不需要用户名密码）
    loginResult = c.start("ForceLogin=1", '', mainCallback)
    if(loginResult.ErrorCode != 0):
        print("login in fail")
        exit()

    # 市场特征
    date = datetime.today().strftime("%Y-%m-%d")
    # 指数涨幅
    zs0=c.css("000985.CSI,000300.SH,000905.SH,000852.SH","NAME,DIFFERRANGE","N=-0,TradeDate="+date+",AdjustFlag=1,Ispandas=1")    
    zs5=c.css("000985.CSI,000300.SH,000905.SH,000852.SH","NAME,DIFFERRANGEN","N=-5,TradeDate="+date+",AdjustFlag=1,Ispandas=1")
    zs10=c.css("000985.CSI,000300.SH,000905.SH,000852.SH","NAME,DIFFERRANGEN","N=-10,TradeDate="+date+",AdjustFlag=1,Ispandas=1")
    zs20=c.css("000985.CSI,000300.SH,000905.SH,000852.SH","NAME,DIFFERRANGEN","N=-20,TradeDate="+date+",AdjustFlag=1,Ispandas=1")
    zs60=c.css("000985.CSI,000300.SH,000905.SH,000852.SH","NAME,DIFFERRANGEN","N=-60,TradeDate="+date+",AdjustFlag=1,Ispandas=1")
    zs120=c.css("000985.CSI,000300.SH,000905.SH,000852.SH","NAME,DIFFERRANGEN","N=-120,TradeDate="+date+",AdjustFlag=1,Ispandas=1")
    zs250=c.css("000985.CSI,000300.SH,000905.SH,000852.SH","NAME,DIFFERRANGEN","N=-250,TradeDate="+date+",AdjustFlag=1,Ispandas=1")
    # 指数 （日）主力净流入资金
    date = (datetime.today() + timedelta(days = -0)).strftime("%Y-%m-%d")
    zszj0=c.css("000985.CSI,000300.SH,000905.SH,000852.SH","NAME,NETINFLOW","TradeDate="+date+",AdjustFlag=1,Ispandas=1")
    date = (datetime.today() + timedelta(days = -1)).strftime("%Y-%m-%d")
    zszj1=c.css("000985.CSI,000300.SH,000905.SH,000852.SH","NAME,NETINFLOW","TradeDate="+date+",AdjustFlag=1,Ispandas=1")
    date = (datetime.today() + timedelta(days = -2)).strftime("%Y-%m-%d")
    zszj2=c.css("000985.CSI,000300.SH,000905.SH,000852.SH","NAME,NETINFLOW","TradeDate="+date+",AdjustFlag=1,Ispandas=1")
    # 指数走势
    date = datetime.today().strftime("%Y-%m-%d")
    zszs=c.csd("000985.CSI,000300.SH,000905.SH,000852.SH","CLOSE","2005-01-01",""+date+"","period=3,adjustflag=3,curtype=1,order=1,market=CNSESH,Ispandas=1")
    zszs3=c.csd("000985.CSI,000300.SH,000905.SH,000852.SH","CLOSE","2020-01-01",""+date+"","period=1,adjustflag=3,curtype=1,order=1,market=CNSESH,Ispandas=1")
    print(zszs)
    # 指数估值
    zspb=c.csd("000985.CSI,000300.SH,000905.SH,000852.SH","PBMRQ","2013-01-01",""+date+"","DelType=1,period=3,adjustflag=1,curtype=1,order=1,market=CNSESH,Ispandas=1")
    zspb3=c.csd("000985.CSI,000300.SH,000905.SH,000852.SH","PBMRQ","2020-01-01",""+date+"","DelType=1,period=1,adjustflag=1,curtype=1,order=1,market=CNSESH,Ispandas=1")
    zspbclose=c.css("000985.CSI,000300.SH,000905.SH,000852.SH","SHORTNAME,PBMRQ","TradeDate="+date+",DelType=1,Ispandas=1")

#退出
    data = logoutResult = c.stop()
except Exception as ee:
    print("error >>>",ee)
    traceback.print_exc()
else:
    print("demo end")

# 指数走势
# 变更列名
zszs.columns=['指数','日期','收盘价']
zszs['指数'] = zszs['指数'].str.replace('000985.CSI','中证全指')
zszs['指数'] = zszs['指数'].str.replace('000300.SH','沪深300')
zszs['指数'] = zszs['指数'].str.replace('000905.SH','中证500')
zszs['指数'] = zszs['指数'].str.replace('000852.SH','中证1000')
fig = px.line(zszs,x='日期', y='收盘价',color='指数')
fig.update_layout(xaxis = dict(tickmode='linear',tick0 = 1,dtick = 24,),width=1200,height=600,title={'text': "指数走势",'y':0.98,'x':0.5,'xanchor': 'center','yanchor': 'top'},
                title_font_size=35,font_size=15,title_font_color='red',xaxis_tickangle=-45,xaxis_title=None,yaxis_title=None)
fig.update_yaxes(type='log')

In [3]:
code = ['中证全指','沪深300','中证500','中证1000']
# 指数走势对比
df=qs.get_price(code,start='2012-01-01').dropna()
print(df)
#plot.line(df/df.iloc[0],title='全球指数累计涨幅(2012-2022)')
fig = px.line(df,x=df.index,y='中证全指')
fig.update_layout(xaxis = dict(tickmode='linear',tick0 = 1,dtick = 250,),width=1200,height=600,title={'text': "A股PB",'y':0.98,'x':0.5,'xanchor': 'center','yanchor': 'top'},
                title_font_size=35,font_size=15,title_font_color='red',xaxis_tickangle=-45,xaxis_title=None,yaxis_title=None)
fig.update_yaxes(type='log')
fig.show()

100%|██████████| 4/4 [00:01<00:00,  3.21it/s]


              沪深300     中证全指    中证500   中证1000
date                                          
2012-01-04  2298.75  2571.95  3192.23  3362.83
2012-01-05  2276.39  2513.70  3074.37  3205.40
2012-01-06  2290.60  2527.25  3087.55  3224.50
2012-01-09  2368.57  2619.64  3212.90  3365.90
2012-01-10  2447.35  2713.53  3345.39  3503.24
...             ...      ...      ...      ...
2022-11-01  3634.17  4634.42  5941.15  6439.82
2022-11-02  3677.81  4692.07  5997.80  6524.51
2022-11-03  3647.90  4683.03  5996.72  6566.84
2022-11-04  3767.17  4808.23  6133.46  6709.64
2022-11-07  3768.77  4812.69  6133.53  6703.55

[2634 rows x 4 columns]


In [ ]:
class mq():
    def zszf(code,days,text):
        # 指数涨幅
        date=xshg_range.index.strftime("%Y-%m-%d").tolist()[-2]
        df1=qs.get_price(code, start=""+date+"", end=""+date+"", freq='d', fqt=1)
        date=xshg_range.index.strftime("%Y-%m-%d").tolist()[days]
        df2=qs.get_price(code, start=""+date+"", end=""+date+"", freq='d', fqt=1)
        # 合并运算
        df=pd.concat([df1,df2],axis=0,ignore_index=True).T
        df['涨幅']=(df[0]/df[1]-1)*100
        # 绘图
        fig = px.bar(df,x=code,y='涨幅',text='涨幅')
        fig.update_traces(texttemplate='%{text:.0f}',textposition='inside',marker=dict(color=np.where(np.array(df['涨幅'])>0,'red','limegreen'))) 
        fig.update_layout(width=1200,height=600,title={'text': text,'y':0.98,'x':0.5,'xanchor': 'center','yanchor': 'top'},title_font_size=45,font_size=35,title_font_color='red',showlegend=False,xaxis_title=None,yaxis_title=None)
        fig.show()
    def bkzf(self,days,text):
        # 板块涨幅
        df=qs.ths_money(self,n=days)
        df['涨幅']=df['涨幅'].str[:-1].astype(float)
        df=df.iloc[0:20].append(df.iloc[-20:])
        # 绘图
        fig = px.bar(df,x=self+'名称',y='涨幅',text='涨幅')
        fig.update_traces(texttemplate='%{text:.0f}',textposition='inside',marker=dict(color=np.where(np.array(df['涨幅'])>0,'red','limegreen'))) 
        fig.update_layout(width=1200,height=600,title={'text': text,'y':0.98,'x':0.5,'xanchor': 'center','yanchor': 'top'},title_font_size=35,font_size=18,title_font_color='red',xaxis_tickangle=-45,showlegend=False,xaxis_title=None,yaxis_title=None)
        fig.show()
    def zszj(code,days,text):
        code = ['中证全指','沪深300','中证500','中证1000']

In [ ]:
# 指数涨幅
code = ['中证全指','沪深300','中证500','中证1000']
mq.zszf(code,-3,"当日指数涨幅%")
mq.zszf(code,-8,"5日指数涨幅%")
mq.zszf(code,-13,"10日指数涨幅%")
mq.zszf(code,-23,"20日指数涨幅%")
mq.zszf(code,-63,"60日指数涨幅%")
mq.zszf(code,-123,"120日指数涨幅%")
mq.zszf(code,-253,"250日指数涨幅%")

In [ ]:
# 风格涨幅
code = ['大盘价值','中盘价值','小盘价值','大盘成长','中盘成长','小盘成长']
mq.zszf(code,-3,"当日风格涨幅%")
mq.zszf(code,-8,"5日风格涨幅%")
mq.zszf(code,-13,"10日风格涨幅%")
mq.zszf(code,-23,"20日风格涨幅%")
mq.zszf(code,-63,"60日风格涨幅%")
mq.zszf(code,-123,"120日风格涨幅%")
mq.zszf(code,-253,"250日风格涨幅%")

In [ ]:
# 行业风格涨幅
code = ['金融','周期','消费','成长','稳定']
mq.zszf(code,-3,"当日行业风格涨幅%")
mq.zszf(code,-8,"5日行业风格涨幅%")
mq.zszf(code,-13,"10日行业风格涨幅%")
mq.zszf(code,-23,"20日行业风格涨幅%")
mq.zszf(code,-63,"60日行业风格涨幅%")
mq.zszf(code,-123,"120日行业风格涨幅%")
mq.zszf(code,-253,"250日行业风格涨幅%")

In [ ]:
# 行业涨幅
mq.bkzf('行业',2,"当日行业前后20涨幅%")
mq.bkzf('行业',5,"5日行业前后20涨幅%")
mq.bkzf('行业',10,"10日行业前后20涨幅%")
mq.bkzf('行业',20,"20日行业前后20涨幅%")

In [ ]:
# 概念涨幅
mq.bkzf('概念',2,"当日概念前后20涨幅%")
mq.bkzf('概念',5,"5日概念前后20涨幅%")
mq.bkzf('概念',10,"10日概念前后20涨幅%")
mq.bkzf('概念',20,"20日概念前后20涨幅%")

In [ ]:
# 资金净流入
import qstock as qs
from datetime import *
import exchange_calendars as xcals
import numpy as np
import pandas as pd
from datetime import timedelta, datetime
import time as _time
from datetime import *
from pandas import DataFrame
import plotly_express as px
import plotly.graph_objects as go

#行业板块10日资金流数据
df=qs.ths_money('行业',n=2)
print(df)
